# Robótica: Programación con python 
    
### Pablo Marcos Arenal 

Este es un breve tutorial de conexión de python a CoppeliaSim para las clases de Robótica del Grado en Informática de la VIU.
Además se presentarán unos simples ejercicios para el manejo de un robot SCARA. 
Este programa servirá de guía para la realización de la *Actividad 3* del curso.

Este tutorial esta basado en el taller [Robopy](https://github.com/cmoralesd/robopy) de Claudio Morales.



## Paso 1: Setup

* Buscar el archivo 'remoteapi.dll' de CoppeliaSim en su árbol de directorios. Deberían estar en el *path*:

> ### Windows:
>```console
>C:\Program Files\CoppeliaRobotics\CoppeliaSimEdu\programming\remoteApiBindings\lib\lib\Windows
>```
>### Mac:
>```console
>[..]\CoppeliaSim_Edu_V4_0_0_Mac/programming/remoteApiBindings/lib/lib/MacOS/remoteapi.dll
>```

* Colocar el archivo *python* (o *Jupyter*; este mismo cuaderno Jupyter *Robotica_Programacion_con_python.ipynb* por ejemplo) en la misma carpeta que el archivo *MTB_Robot.ttt* y los archivos 'sim.py', 'simConst.py' y 'remoteapi.dll'.



## Paso 2:  CoppeliaSim
* Abrir la escena *MTB_Robot.ttt* en CoppeliaSim


* Aceptar las conexiones entrantes
![Conexiones entrantes](Figures/conexiones_entrantes.png)


* Editar un script (el de la base del robot, preferiblemente) con *simRemoteApi.start(19999)*

```console

function sysCall_init() 
   simRemoteApi.start(19999)
end

function sysCall_cleanup() 
end 

function sysCall_actuation() 
end 
```

* Iniciar la simulación.

## Paso 3: Jupyter Notebook
* Abrir el archivo *Robotica_Programacion_con_python.ipynb* en *Jupyter*.

* Trust: Confiar en el notebook

### Importamos las librerías necesarias


In [30]:
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico
import time
import numpy as np

### Conexión con CoppeliaSim
La función *connect(port)* establece la conexión con CoppeliaSim. Utilizaremos las funciones del API Remoto de CoppeliaSim.

Más información en:
http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

*port* debe coincidir con el puerto de conexión en CoppeliaSim, 19999 en nuestro caso.
Devuelve el número de cliente o -1 si no puede establecer conexión.



In [31]:
def connect(port):
# Establece la conexión con CoppeliaSim 
# port debe coincidir con el puerto de conexión en CoppeliaSim, 19999 en nuestro caso
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("Conectado al puerto: ", port)
    else: print("WARNING: no se pudo conectar")
    return clientID


Llamar a la función para conectarse al servidor de CoppeliaSim.
Es necesario hacerlo cada vez que se reinicia la simulación.


In [32]:
clientID = connect(19999)

Conectado al puerto:  19999


## Control del robot
Mediante el uso de manejadores (o "handlers") se controlarán cada uno de los elementos del robot. Principalmente las articulaciones. 

> 📘
**NOTA:** Añadir un "Dummy" al efector final.

### Manejadores ("handlers)
A cada elemento de la escena que se quiere manejar se le asocia un "handler" mediante un identificador. Con él se hace referencia a ese elemento en el código. En este caso el "handler" del "Dummy" que hemos colocado en el efector final. 

In [33]:
# Obtenemos el handler del Dummy
retCode,end_effector=sim.simxGetObjectHandle(clientID,'Dummy',sim.simx_opmode_blocking)

print(end_effector)

0


Mediante los "handlers" y las funciones específicas de la librería sim se controlan los elementos del robot.
### Posición y orientación:

In [34]:
# posición actual del efector final
retCode,pos=sim.simxGetObjectPosition(clientID, end_effector, -1, sim.simx_opmode_blocking)

print(pos)

[1.4203234910964966, -0.7421712875366211, 0.48568829894065857]


In [35]:
# orientación actual del efector final
retCode,orient=sim.simxGetObjectOrientation(clientID, end_effector, -1, sim.simx_opmode_blocking)
print(orient)

[-1.826035737991333, -0.7889206409454346, 2.9585115909576416]


### Movimiento de las articulaciones

In [36]:
# Obtenemos el handler de las articulaciones
retCode,joint1=sim.simxGetObjectHandle(clientID,'MTB_joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'MTB_joint3',sim.simx_opmode_blocking)


# fijamos las posiciones a las que queremos llevar cada articulación 
q1 = 0 * np.pi/180

# y usamos las funciones correspondientes para mover el robot
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q1, sim.simx_opmode_oneshot)


In [37]:
# fijamos la posicion a las que queremos llevar joint2
q2 = 0 * np.pi/180

# y movemos la articulación
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q2, sim.simx_opmode_oneshot)


In [38]:
# fijamos la posicion a las que queremos llevar joint3
q3 = 0.0 #0.1

# y movemos el brazo
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q3, sim.simx_opmode_oneshot)

### Setup del robot en la posición inicial


In [39]:
# Enviamos el robot a la posición inicial
q = [0, 0, 0]

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


### Posición y orientación de un objeto externo

In [40]:
# Obtenemos el handler de la Caja1
retCode,caja1=sim.simxGetObjectHandle(clientID,'Caja1',sim.simx_opmode_blocking)

print("Handler: ",caja1)

retCode,pos=sim.simxGetObjectPosition(clientID, caja1, -1, sim.simx_opmode_blocking)

print('Posición: ',pos)

Handler:  35
Posición:  [0.540294885635376, -0.04687117040157318, 0.02499992400407791]


## Cinemática Directa
A partir de los ángulos que se quieran dar a las articulaciones, determinamos la posición del efector final. Tal como vimos en clase (Tema 3), calculamos la posición del actuador final a partir de las ecuaciones del modelo geométrico:

![Modelo geometrico Robot SCARA](Figures/SCARA_geometric_model.png)

A partir de este modelo determinamos la matriz de transformación.

Las longitudes de los brazos en este ejemplo son:

```console
l1 = 0.234
l2 = 0.467
l3 = 0.4005
```

In [41]:
# Matriz de transformación directa

l1 = 0.234
l2 = 0.467
l3 = 0.4005

q1 = sp.symbols('q1') # ángulo de la articulación rotacional joint1, en radianes
q2 = sp.symbols('q2') # ángulo de la articulación rotacional joint2, en radianes
q3 = sp.symbols('q3') # posición de la articulación prismática joint3, en metros
q4 = sp.symbols('q4') # ángulo de la articulación rotacional joint4, en radianes

T = sp.Matrix([[sp.cos(q1 + q2 + q4), -sp.sin(q1 + q2 + q4), 0, l2*sp.cos(q1) + l3*sp.cos(q1 + q2)],
            [sp.sin(q1 + q2 + q4), sp.cos(q1 + q2 + q4), 0, l2*sp.sin(q1) + l3*sp.sin(q1 + q2)], 
            [0, 0, 1, l1 - q3], 
            [0, 0, 0, 1]])
T

Matrix([
[cos(q1 + q2 + q4), -sin(q1 + q2 + q4), 0, 0.467*cos(q1) + 0.4005*cos(q1 + q2)],
[sin(q1 + q2 + q4),  cos(q1 + q2 + q4), 0, 0.467*sin(q1) + 0.4005*sin(q1 + q2)],
[                0,                  0, 1,                         -q3 + 0.234],
[                0,                  0, 0,                                   1]])

### Cálculo de la matriz de rotación

La matriz de rotación se puede calcular a partir de los ángulos de Euler (que son los que nos da CoppeliaSim) con las ecuaciones que vimos en el Tema 3:

![Posiciones a partir de los angulos Euler](Figures/angulos_Euler.png)

In [42]:
# Definimos una función para construir la matriz de rotación
# a partir de los ángulos de Euler

def matrixFromEuler(alpha, beta, gamma):
    # angulos en radianes
    Ra = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    Rb = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta), 0],
                   [0, 1, 0, 0],
                   [-sp.sin(beta), 0, sp.cos(beta), 0],
                   [0, 0, 0, 1]])
    Rc = sp.Matrix([[sp.cos(gamma), -sp.sin(gamma), 0, 0],
                   [sp.sin(gamma), sp.cos(gamma), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    T = Ra*Rb*Rc
    return T

In [43]:

matrixFromEuler(45*sp.pi/180, 0, 45*sp.pi/180)

Matrix([
[sqrt(2)/2, -sqrt(2)/2,          0, 0],
[      1/2,        1/2, -sqrt(2)/2, 0],
[      1/2,        1/2,  sqrt(2)/2, 0],
[        0,          0,          0, 1]])

###  Posición y orientación del punto de destino


In [15]:
# A partir de las coordenadas de posición y los ángulos de Euler
# es posible obtener la matriz de transformación

x = 0.5
y = 0.5
z = 0.1
alpha = 0
beta = 0
gamma = 45*sp.pi/180

t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)
D

Matrix([
[sqrt(2)/2, -sqrt(2)/2, 0, 0.5],
[sqrt(2)/2,  sqrt(2)/2, 0, 0.5],
[        0,          0, 1, 0.1],
[        0,          0, 0,   1]])

## Cálculo de la cinemática inversa
A partir de esta matriz de transformación directa (T) y la posición final (D) calculamos la cinemática inversa.


In [16]:
# La función nsolve de sympy busca valores busca soluciones que
# igualan una expresión a cero. Si requerimos T = D, entonces buscamos T-D = 0

T-D


Matrix([
[cos(q1 + q2 + q4) - sqrt(2)/2, -sin(q1 + q2 + q4) + sqrt(2)/2, 0, 0.467*cos(q1) + 0.4005*cos(q1 + q2) - 0.5],
[sin(q1 + q2 + q4) - sqrt(2)/2,  cos(q1 + q2 + q4) - sqrt(2)/2, 0, 0.467*sin(q1) + 0.4005*sin(q1 + q2) - 0.5],
[                            0,                              0, 0,                               -q3 + 0.134],
[                            0,                              0, 0,                                         0]])

In [17]:
# Buscamos una solución relajando la precisión requerida para facilitar el cálculo.
# La precisión por defecto es de 15 cifras significativas.

try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('No se encontró la solución')
    q = [0, 0, 0, 0]
q

Matrix([
[ 0.220089],
[  1.23996],
[    0.134],
[-0.674648]])

### Setup del "suction pad"
El robot cuenta con un "suction pad" en el extremo. Éste se activa automáticamente al acercarse al cubo (1 mm o menos) y se desactiva cuando el cubo se coloca sobre la plataforma (5 mm o menos).

In [18]:
def setEffector(val):
# acciona el efector final
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

In [19]:
# Activamos o desactivamos el actuador final
setEffector(1)

0

### Iniciamos de nuevo la simulación en MTB_Pick_N_Place

In [20]:
clientID = connect(19999)

retCode,effector=sim.simxGetObjectHandle(clientID,'effector',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'MTB_joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'MTB_joint3',sim.simx_opmode_blocking)
retCode,joint4=sim.simxGetObjectHandle(clientID,'MTB_joint4',sim.simx_opmode_blocking)
retCode,caja1=sim.simxGetObjectHandle(clientID,'Caja1',sim.simx_opmode_blocking)
retCode,caja2=sim.simxGetObjectHandle(clientID,'Caja2',sim.simx_opmode_blocking)
retCode,caja3=sim.simxGetObjectHandle(clientID,'Caja3',sim.simx_opmode_blocking)

Conectado al puerto:  19999


In [21]:
# enviamos los ángulos a las articulaciones

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)


In [22]:
# verificamos la posición del actuador
retCode,pos=sim.simxGetObjectPosition(clientID, effector, -1, sim.simx_opmode_blocking)
print(pos)

[0.4999621510505676, 0.49999555945396423, 0.09972193837165833]


In [23]:
# y la orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, effector, -1, sim.simx_opmode_blocking)
print(eul[0]*180/np.pi)
print(eul[1]*180/np.pi)
print(eul[2]*180/np.pi)

0.0034169870193165117
0.0035818439152365844
45.00053742208893


## Movemos la caja 

1. Obtenemos la posición y orientación de la caja (las coordenadas que obtendrá serán las coordenadas del centro del objeto).
2. Vamos hasta ella.
3. Activamos el efector final con la función setEffector()
4. Nos movemos hasta la posición final (la única orientación que puede ser manejada es el ángulo gamma; rotación respecto al eje z, respecto del sistema de coordenadas del mundo).
5. Desactivamos el efector final.




In [29]:
# Para el primer cubo
clientID = connect(19999)

# obtenemos la posición
print("leyendo la posición objetivo...")
retCode,pos=sim.simxGetObjectPosition(clientID, caja1, -1, sim.simx_opmode_blocking)
print(pos)
# y orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, caja1, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

# definimos las coordenadas de destino
x = pos[0]
y = pos[1]
z = pos[2] + 0.026 # distancia del centro al borde + tolerancia
alpha = eul[0]
beta = eul[1]
gamma = eul[2]
t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)

# calculamos la cinemática inversa
print("calculando la cinemática inversa...")
try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
print (q)

# movemos el robot a la posición
print("moviendo a la posición objetivo...")
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# activamos el efector
setEffector(1)
time.sleep(1)
# levantamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# volvemos a la posición inicial
retCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el cubo
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# desactivamos el actuador
time.sleep(1)
setEffector(0)
# y levantamos
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
print("movimiento concluido!")

Conectado al puerto:  19999
leyendo la posición objetivo...
[0.8615366220474243, 9.232490992872044e-05, 0.02499992400407791]
-3.422827338597573e-08
6.227469266007323e-07
-12.989916231816192
calculando la cinemática inversa...
no se encontró la solución
[0, 0, 0, 0]
moviendo a la posición objetivo...
movimiento concluido!


# Ejercicio examen


In [27]:
r_u = -5
r_v = -7
r_w = 9

r_uvw = sp.Matrix([r_u,r_v,r_w,1])

t_x = 5
t_y = -6
t_z = 14


t = sp.Matrix([[0, -1, 0, t_x],
               [1, 0, 0,  t_y], 
               [0, 0, 1,  t_z], 
               [0, 0, 0, 1]])

t

Matrix([
[0, -1, 0,  5],
[1,  0, 0, -6],
[0,  0, 1, 14],
[0,  0, 0,  1]])

In [28]:
sol = t*r_uvw
sol


Matrix([
[ 12],
[-11],
[ 23],
[  1]])